In [1]:
import sys
package_path = '../EfficientNet-PyTorch/'
sys.path.append(package_path)

from efficientnet_pytorch import EfficientNet
from radam import RAdam, PlainRAdam, AdamW

In [2]:
from transforms import (pre_transforms, post_transforms, my_transforms, Compose)

In [3]:
import os
import gc
import time 
import numpy as np 
import pandas as pd
import cv2
from PIL import Image
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as D
import torch.nn.functional as F
from torchvision import transforms as T
from torch.optim.lr_scheduler import ExponentialLR, StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torch.utils.data.sampler import SubsetRandomSampler

from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers.tqdm_logger import ProgressBar
from ignite.handlers import  EarlyStopping, ModelCheckpoint

import albumentations
from albumentations import torch as AT

#from am_softmax import AMSoftmaxLoss, AngleSimpleLinear

import warnings
warnings.filterwarnings('ignore')

## Config

In [4]:
config = {
    'SEED': 42,
    'CLASSES': 1,
    'PATH_DATA': '',
    'PRETRAIN_PATH_DATA': 'pretrain_2015_data',
    'DEVICE': 'cuda',
    'BATCH_SIZE': 8,
    'VAL_SIZE': 0.1,
    'MODEL_NAME': 'EffNet_b3_Adam_regr',
    'USE_ANGULAR': False,
    'USE_BN': False,
    'LR': 1e-4,
    'TURN_OFF_ON_N_EPOCHS': 0,
}

image_size = 256
upsampling = True
crop_from_gray = True
circle_crop = True
normalize = True
ben_preprocess = 10
hor_flip = 0.5
ver_flip = 0.33
rotate = 360
random_scale = 0.35
random_scale_p = 0.75
brightness = 0.35
contrast = 0.35
color_p = 0.5

In [5]:
def seed_torch(seed=42):
    import random; import os

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(config['SEED'])

In [6]:
def argmax_predictions(predictions):
    coef = [0.57, 1.57, 2.57, 3.57]

    for i, pred in enumerate(predictions):
        if pred < coef[0]:
            predictions[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            predictions[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            predictions[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            predictions[i] = 3
        else:
            predictions[i] = 4
            
    return predictions

## Model

In [14]:
class EffNet(nn.Module):
    def __init__(self, num_classes=1000, num_channels=3, use_angular=False, use_bn=False):
        super().__init__()
        self.use_angular = use_angular
        self.use_bn = use_bn
        self.bn = nn.BatchNorm2d(num_channels)
        
        self.features = EfficientNet.from_pretrained('efficientnet-b3', num_classes=num_classes)
        #print(self.features)
        
        if self.use_angular:
            self.features._fc = AngleSimpleLinear(1280, num_classes)
        
    def forward(self, x):
        if self.use_bn:
            x = self.bn(x)
            
        out = self.features(x)
        return out

## PreTrain

In [8]:
pretrain = pd.read_csv(config['PRETRAIN_PATH_DATA']+'/trainLabels.csv')
pretrain_cropped = pd.read_csv(config['PRETRAIN_PATH_DATA']+'/trainLabels_cropped.csv')

In [9]:
len(pretrain), len(pretrain_cropped)

(35126, 35108)

In [10]:
class PreTrainGlassDataset(D.Dataset):
    def __init__(self, df, transform=T.Compose([T.CenterCrop(32), T.ToTensor()]), y=None):
        self.df = df
        self.image_files_list = [config['PRETRAIN_PATH_DATA'] + f'/resized_train/{i}.jpeg' for i in df['image'].values]
        self.labels = y            
        self.transform = transform

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        final = self.transform(image=image)
        final = final['image']
        
        label = self.labels[idx]
        
        return final, label

In [11]:
pre_transform_fn = pre_transforms(image_size=image_size, 
                                  crop_from_gray=crop_from_gray, 
                                  circle_crop=circle_crop,
                                  ben_preprocess=ben_preprocess,
                                  random_scale=random_scale, 
                                  random_scale_p=random_scale_p,
                                  brightness=brightness,
                                  contrast=contrast,
                                  color_p=color_p
                                  )

my_transform_fn = my_transforms(hor_flip=hor_flip,
                                ver_flip=ver_flip,
                                rotate=rotate)

post_transform_fn = post_transforms(normalize=normalize)

data_transforms = Compose([pre_transform_fn, my_transform_fn, post_transform_fn])

In [12]:
dataset = PreTrainGlassDataset(df=pretrain, transform=data_transforms, y=pretrain.level)

tr, val = train_test_split(pretrain.level, stratify=pretrain.level,
                           test_size=config['VAL_SIZE'], random_state=config['SEED'])
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))

train_loader = torch.utils.data.DataLoader(dataset, batch_size=config['BATCH_SIZE'],
                                           sampler=train_sampler, num_workers=4)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=config['BATCH_SIZE'],
                                           sampler=valid_sampler, num_workers=4)

In [13]:
model = EffNet(num_classes=config['CLASSES'], use_angular=False, use_bn=False)
model.to(config['DEVICE']);

Loaded pretrained weights for efficientnet-b3


In [14]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [15]:
valid_loss_min = np.Inf
patience = 5

# current number of epochs, where validation loss didn't increase
p = 0
# whether training should be stopped
stop = False

# number of epochs to train the model
n_epochs = 20
for epoch in range(1, n_epochs+1):
    
    model.train()
    train_loss = []
    train_auc = []
    for (data, target) in tqdm_notebook(train_loader):

        data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())
        
        a = target.data.cpu().numpy()
        b = output[:,-1].detach().cpu().numpy()
        # train_auc.append(roc_auc_score(a, b))
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_loss = []
    val_auc = []
    for (data, target) in (valid_loader):
        data, target = data.cuda(), target.cuda()
        output = model(data)

        loss = criterion(output, target.float())

        val_loss.append(loss.item()) 
        a = target.data.cpu().numpy()
        b = output[:,-1].detach().cpu().numpy()
        # val_auc.append(roc_auc_score(a, b))

    # print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train auc: {np.mean(train_auc):.4f}, valid auc: {np.mean(val_auc):.4f}')
    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f} ')
    
    valid_loss = np.mean(val_loss)
    # scheduler.step(valid_loss)
    if valid_loss <= valid_loss_min:
        torch.save(model.state_dict(), '{0}/{0}_{1}_pretrain.pt'.format(config['MODEL_NAME'], epoch))
        valid_loss_min = valid_loss
        p = 0

    # check if validation loss didn't improve
    if valid_loss > valid_loss_min:
        p += 1
        print(f'{p} epochs of increasing val loss')
        if p > patience:
            print('Stopping training')
            stop = True
            break        
            
    if stop:
        break


Epoch 1, train loss: 0.9533, valid loss: 0.9419 



Epoch 2, train loss: 0.9482, valid loss: 0.9444 
1 epochs of increasing val loss



Epoch 3, train loss: 0.9460, valid loss: 0.9403 



Epoch 4, train loss: 0.9456, valid loss: 0.9520 
1 epochs of increasing val loss



Epoch 5, train loss: 0.9454, valid loss: 0.9412 
2 epochs of increasing val loss



Epoch 6, train loss: 0.9450, valid loss: 0.9410 
3 epochs of increasing val loss



Epoch 7, train loss: 0.9442, valid loss: 0.9425 
4 epochs of increasing val loss


KeyboardInterrupt: 

## Data

In [7]:
config = {
    'SEED': 42,
    'CLASSES': 1,
    'PATH_DATA': '',
    'PRETRAIN_PATH_DATA': 'pretrain_2015_data',
    'DEVICE': 'cuda',
    'BATCH_SIZE': 8,
    'VAL_SIZE': 0.1,
    'MODEL_NAME': 'EffNet_b3_Adam_regr',
    'USE_ANGULAR': False,
    'USE_BN': False,
    'LR': 1e-4,
    'TURN_OFF_ON_N_EPOCHS': 0,
}

image_size = 256
upsampling = False
crop_from_gray = True
circle_crop = True
normalize = True
ben_preprocess = 10
hor_flip = 0.5
ver_flip = 0.33
rotate = 360
random_scale = 0.35
random_scale_p = 0.75
brightness = 0.35
contrast = 0.35
color_p = 0.5

In [8]:
train = pd.read_csv(config['PATH_DATA']+'train.csv')
test = pd.read_csv(config['PATH_DATA']+'test.csv')
sample_submission = pd.read_csv(config['PATH_DATA']+'sample_submission.csv')

In [9]:
len(train), len(test)

(3662, 1928)

In [10]:
class GlassDataset(D.Dataset):
    def __init__(self, df, datatype='train',
                 transform=T.Compose([T.CenterCrop(32), T.ToTensor()]), y=None):
        self.df = df
        self.datatype = datatype
        self.image_files_list = [f'{self.datatype}_images/{i}.png' for i in df['id_code'].values]
        
        if self.datatype == 'train':
            self.labels = y
        else:
            self.labels = np.zeros((df.shape[0], 1))
            
        self.transform = transform

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        final = self.transform(image=image)
        final = final['image']
        
        label = self.labels[idx]
        
        return final, label

---

In [11]:
pre_transform_fn = pre_transforms(image_size=image_size, 
                                  crop_from_gray=crop_from_gray, 
                                  circle_crop=circle_crop,
                                  ben_preprocess=ben_preprocess,
                                  random_scale=random_scale, 
                                  random_scale_p=random_scale_p,
                                  brightness=brightness,
                                  contrast=contrast,
                                  color_p=color_p
                                  )

my_transform_fn = my_transforms(hor_flip=hor_flip,
                                ver_flip=ver_flip,
                                rotate=rotate)

post_transform_fn = post_transforms(normalize=normalize)

data_transforms = Compose([pre_transform_fn, my_transform_fn, post_transform_fn])

In [12]:
dataset = GlassDataset(df=train, datatype='train', transform=data_transforms, y=train.diagnosis)
#test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)

tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis,
                           test_size=config['VAL_SIZE'], random_state=config['SEED'])
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))

train_loader = torch.utils.data.DataLoader(dataset, batch_size=config['BATCH_SIZE'],
                                           sampler=train_sampler, num_workers=4)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=config['BATCH_SIZE'],
                                           sampler=valid_sampler, num_workers=4)
#test_loader = torch.utils.data.DataLoader(test_set, batch_size=config['BATCH_SIZE'],
#                                          num_workers=4)

In [15]:
model = EffNet(num_classes=config['CLASSES'], use_angular=False)
model.load_state_dict(torch.load('{0}/{0}_3_pretrain.pt'.format('EffNet_b3_Adam_regr')))
model.to(config['DEVICE']);

Loaded pretrained weights for efficientnet-b3


In [16]:
#for name, child in model.named_children():
#    if name == 'bn':
#        print(name + ' is unfrozen')
#        for param in child.parameters():
#            param.requires_grad = True
            
#    else:
#        for child_name, child_2 in child.named_children():
#            if child_name == '_fc':
#                print(child_name + ' is unfrozen')
#                for param in child_2.parameters():
#                    param.requires_grad = True
#            else:
#                for param in child_2.parameters():
#                    param.requires_grad = False

## Training

In [17]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
#scheduler = StepLR(optimizer, step_size=2, gamma=0.1)
#optimizer = optim.SGD(model.parameters(), lr=3e-2, momentum=0.95)
#scheduler = CyclicLR(optimizer, base_lr=lr, max_lr=0.01, step_size=5, mode='triangular2')
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=2, )

In [18]:
valid_loss_min = np.Inf
patience = 5

# current number of epochs, where validation loss didn't increase
p = 0
# whether training should be stopped
stop = False

# number of epochs to train the model
n_epochs = 20
for epoch in range(1, n_epochs+1):
    
    model.train()
    train_loss = []
    train_auc = []
    for (data, target) in tqdm_notebook(train_loader):

        data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())
        
        a = target.data.cpu().numpy()
        b = output[:,-1].detach().cpu().numpy()
        # train_auc.append(roc_auc_score(a, b))
        loss.backward()
        optimizer.step()
        
    del data, target, output
    torch.cuda.empty_cache()
    
    model.eval()
    val_loss = []
    val_auc = []
    for (data, target) in (valid_loader):
        data, target = data.cuda(), target.cuda()
        output = model(data)

        loss = criterion(output, target.float())

        val_loss.append(loss.item()) 
        a = target.data.cpu().numpy()
        b = output[:,-1].detach().cpu().numpy()
        # val_auc.append(roc_auc_score(a, b))
        
    del data, target, output
    torch.cuda.empty_cache()

    # print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train auc: {np.mean(train_auc):.4f}, valid auc: {np.mean(val_auc):.4f}')
    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f} ')
    
    valid_loss = np.mean(val_loss)
    # scheduler.step(valid_loss)
    if valid_loss <= valid_loss_min:
        torch.save(model.state_dict(), '{0}/{0}_{1}.pt'.format(config['MODEL_NAME'], epoch))
        valid_loss_min = valid_loss
        p = 0

    # check if validation loss didn't improve
    if valid_loss > valid_loss_min:
        p += 1
        print(f'{p} epochs of increasing val loss')
        if p > patience:
            print('Stopping training')
            stop = True
            break        
            
    if stop:
        break


Epoch 1, train loss: 1.7047, valid loss: 1.7397 



Epoch 2, train loss: 1.6958, valid loss: 1.6964 



Epoch 3, train loss: 1.6988, valid loss: 1.7150 
1 epochs of increasing val loss



Epoch 4, train loss: 1.6979, valid loss: 1.7180 
2 epochs of increasing val loss



Epoch 5, train loss: 1.6951, valid loss: 1.7213 
3 epochs of increasing val loss


...


Epoch 8, train loss: 1.6955, valid loss: 1.6971 
2 epochs of increasing val loss



Epoch 9, train loss: 1.6977, valid loss: 1.6975 
3 epochs of increasing val loss



Epoch 10, train loss: 1.6962, valid loss: 1.7044 
4 epochs of increasing val loss



Epoch 11, train loss: 1.6986, valid loss: 1.7070 
5 epochs of increasing val loss



Epoch 12, train loss: 1.6914, valid loss: 1.6945 
6 epochs of increasing val loss
Stopping training
